In [1]:
# -*- coding:utf-8 -*-
'''
功能：通用的模型内部模块，如参数初始化、embedding-pooling、target-attention、multi-heads self-attention等
'''
import sys

import numpy as np
import tensorflow as tf


def emb_init(name, feat_num, embedding_size, zero_first_row=True, pre_trained=False, trained_emb_path=None):
    if not pre_trained:
        with tf.variable_scope("weight_matrix"):
            embeddings = tf.get_variable(name=name,
                                         dtype=tf.float32,
                                         shape=(feat_num, embedding_size),
                                         initializer=tf.contrib.layers.xavier_initializer())

        if zero_first_row:  # The first row of initialization is zero
            embeddings = tf.concat((tf.zeros(shape=[1, embedding_size]), embeddings[1:]), 0)
    else:
        pass
        with tf.variable_scope("pre-trained_weight_matrix"):
            load_emb = np.load(tf.gfile.GFile(trained_emb_path, "rb"))
            embeddings = tf.constant(load_emb, dtype=tf.float32, name=name)
            sys.stdout.flush()

    return embeddings


def nonzero_reduce_mean(emb):  # nonzero-mean-pooling
    axis_2_sum = tf.reduce_sum(emb, axis=2)
    multi_cate_nonzero = tf.count_nonzero(axis_2_sum, 1, keepdims=True, dtype=float)
    multi_cate_sum = tf.reduce_sum(emb, axis=1)
    reduce_mean_emb = tf.div_no_nan(multi_cate_sum, multi_cate_nonzero)
    return reduce_mean_emb


class InteractingLayer:  # multi-heads self-attention
    def __init__(self, num_layer, w_name="default", att_emb_size=32, seed=2020, head_num=3, use_res=1):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.num_layer = num_layer
        self.att_emb_size = att_emb_size
        self.seed = seed
        self.head_num = head_num
        self.use_res = use_res
        self.w_name = w_name

    def __call__(self, inputs):
        input_shape = inputs.get_shape().as_list()
        if len(input_shape) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % len(input_shape))

        embedding_size = int(input_shape[-1])
        self.w_query = tf.get_variable(name=str(self.w_name) + str(self.num_layer) + '_query',
                                       dtype=tf.float32,
                                       shape=(embedding_size, self.att_emb_size * self.head_num),
                                       initializer=tf.contrib.layers.xavier_initializer(seed=self.seed))
        self.w_key = tf.get_variable(name=str(self.w_name) + str(self.num_layer) + '_key',
                                     dtype=tf.float32,
                                     shape=(embedding_size, self.att_emb_size * self.head_num),
                                     initializer=tf.contrib.layers.xavier_initializer(seed=self.seed + 1))
        self.w_value = tf.get_variable(name=str(self.w_name) + str(self.num_layer) + '_value',
                                       dtype=tf.float32,
                                       shape=(embedding_size, self.att_emb_size * self.head_num),
                                       initializer=tf.contrib.layers.xavier_initializer(seed=self.seed + 2))
        if self.use_res:
            self.w_res = tf.get_variable(name=str(self.w_name) + str(self.num_layer) + '_res',
                                         dtype=tf.float32,
                                         shape=(embedding_size, self.att_emb_size * self.head_num),
                                         initializer=tf.contrib.layers.xavier_initializer(seed=self.seed))

        querys = tf.tensordot(inputs, self.w_query, axes=1)  # None F D*head_num
        keys = tf.tensordot(inputs, self.w_key, axes=1)
        values = tf.tensordot(inputs, self.w_value, axes=1)

        # head_num None F D
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))

        inner_product = tf.matmul(querys, keys, transpose_b=True)  # head_num None F F
        # Scale
        inner_product = inner_product / (keys.get_shape().as_list()[-1] ** 0.5)
        # Activation
        self.normalized_att_scores = tf.nn.softmax(inner_product)

        result = tf.matmul(self.normalized_att_scores, values)  # head_num None F D
        result = tf.concat(tf.split(result, self.head_num, axis=0), axis=-1)  # 1 None F D*head_num
        result = tf.squeeze(result, axis=0)  # None F D*head_num

        if self.use_res:
            result += tf.tensordot(inputs, self.w_res, axes=1)
        result = tf.nn.relu(result)

        return result


def attention(queries, keys, keys_length):  # target-attention
    """
      queries:     [B, H] 前面的B代表的是batch_size，H代表向量维度。
      keys:        [B, T, H] T是一个batch中，当前特征最大的长度，每个样本代表一个样本的特征
      keys_length: [B]
    """
    # H 每个query词的隐藏层神经元是多少，也就是H
    queries_hidden_units = queries.get_shape().as_list()[-1]
    # tf.tile为复制函数，1代表在B上保持一致，tf.shape(keys)[1] 代表在H上复制这么多次, 那么queries最终shape为(B, H*T)
    queries = tf.tile(queries, [1, tf.shape(keys)[1]])
    # queries.shape(B, T, H) 其中每个元素(T,H)代表T行H列，其中每个样本中，每一行的数据都是一样的
    queries = tf.reshape(queries, [-1, tf.shape(keys)[1], queries_hidden_units])
    # 下面4个变量的shape都是(B, T, H)，按照最后一个维度concat，所以shape是(B, T, H*4), 在这块就将特征中的每个item和目标item连接在了一起
    din_all = tf.concat([queries, keys, queries - keys, queries * keys], axis=-1)
    # (B, T, 80)
    d_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)
    # (B, T, 40)
    d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)
    # (B, T, 1)
    d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=None, name='f3_att', reuse=tf.AUTO_REUSE)
    # (B, 1, T)
    # 每一个样本都是 [1,T] 的维度，和原始特征的维度一样，但是这时候每个item已经是特征中的一个item和目标item混在一起的数值了
    d_layer_3_all = tf.reshape(d_layer_3_all, [-1, 1, tf.shape(keys)[1]])
    outputs = d_layer_3_all
    # Mask，每一行都有T个数字，keys_length长度为B，假设第1 2个数字是5,6，那么key_masks第1 2行的前5 6个数字为True
    key_masks = tf.sequence_mask(keys_length, tf.shape(keys)[1])  # [B, T]
    key_masks = tf.expand_dims(key_masks, 1)  # [B, 1, T]
    # 创建一个和outputs的shape保持一致的变量，值全为1，再乘以(-2 ** 32 + 1)，所以每个值都是(-2 ** 32 + 1)
    paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
    outputs = tf.where(key_masks, outputs, paddings)  # [B, 1, T]

    # Scale
    outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)  # T，根据特征数目来做拉伸
    # Activation
    outputs = tf.nn.softmax(outputs)  # [B, 1, T]
    # Weighted sum
    outputs = tf.matmul(outputs, keys)  # [B, 1, H]
    return outputs


def attention_multi(queries, keys, keys_length):  # target-attention for multi-feats-queries
    """
      queries:     [B, N, H] (e.g. N is the number of ads)
      keys:        [B, T, H]
      keys_length: [B]
    """
    # H 每个query词的隐藏层神经元是多少，也就是H
    queries_hidden_units = queries.get_shape().as_list()[-1]
    # N
    queries_nums = queries.get_shape().as_list()[1]
    queries = tf.tile(queries, [1, 1, tf.shape(keys)[1]])
    queries = tf.reshape(queries, [-1, queries_nums, tf.shape(keys)[1], queries_hidden_units])  # shape : [B, N, T, H]
    max_len = tf.shape(keys)[1]
    keys = tf.tile(keys, [1, queries_nums, 1])
    keys = tf.reshape(keys, [-1, queries_nums, max_len, queries_hidden_units])  # shape : [B, N, T, H]
    din_all = tf.concat([queries, keys, queries - keys, queries * keys], axis=-1)
    d_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)
    d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)
    d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=None, name='f3_att', reuse=tf.AUTO_REUSE)  # [B, N, T, 1]
    d_layer_3_all = tf.reshape(d_layer_3_all, [-1, queries_nums, 1, max_len])  # [B, N, 1, T]
    outputs = d_layer_3_all
    # Mask
    key_masks = tf.sequence_mask(keys_length, max_len)  # [B, T]
    key_masks = tf.tile(key_masks, [1, queries_nums])  # [B, N, T]
    key_masks = tf.reshape(key_masks, [-1, queries_nums, 1, max_len])  # shape : [B, N, 1, T]
    paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
    outputs = tf.where(key_masks, outputs, paddings)  # [B, N, 1, T]

    # Scale
    outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)

    # Activation
    outputs = tf.nn.softmax(outputs)  # [B, N, 1, T]
    # Weighted sum
    outputs = tf.matmul(outputs, keys)  # [B, N, 1, H]

    # Pooling
    # outputs = tf.reduce_mean(outputs, axis=1)  # [B, 1, H] mean-pooling
    outputs = tf.reduce_sum(outputs, axis=1)  # [B, 1, H] sum-pooling
    return outputs

In [2]:
# -*- coding:utf-8 -*-
'''
功能: 模型共用模块，如训练、预测、保存等
'''

import tensorflow as tf


def model_optimizer(params, mode, labels, out):
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=out)

    else:
        labels = tf.identity(labels, name='labels')
        auc = tf.metrics.auc(labels=labels, predictions=out, name='auc')
        metrics = {
            'auc': auc
        }
        loss = tf.reduce_mean(tf.losses.log_loss(labels=labels, predictions=out))

        # ------bulid optimizer------
        if params.optimizer == 'Adam':
            optimizer = tf.train.AdamOptimizer(learning_rate=params.learning_rate)
        elif params.optimizer == 'Adagrad':
            optimizer = tf.train.AdagradOptimizer(learning_rate=params.learning_rate)
        elif params.optimizer == 'Momentum':
            optimizer = tf.train.MomentumOptimizer(learning_rate=params.learning_rate, momentum=0.95)
        elif params.optimizer == 'ftrl':
            optimizer = tf.train.FtrlOptimizer(learning_rate=params.learning_rate)
        elif params.optimizer == 'Adadelta':
            optimizer = tf.train.AdadeltaOptimizer(learning_rate=params.learning_rate)

        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        else:
            train_op = None

        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            eval_metric_ops=metrics,
            train_op=train_op)


def model_save_pb(params, model):
    features = {
        'cont_feats': tf.FixedLenFeature(dtype=tf.float32, shape=[params.cont_field_count]),
        'cate_feats': tf.FixedLenFeature(dtype=tf.int64, shape=[params.cate_field_count]),
    }
    if params.multi_feats_type in ['dense', 'sparse2dense']:
        for field_name in params.multi_cate_field_list:
            features[field_name[0]] = tf.FixedLenFeature(field_name[1], tf.int64)
    elif params.multi_feats_type == 'sparse':
        for field_name in params.multi_cate_field_list:
            features[field_name[0]] = tf.VarLenFeature(tf.int64)

    if params.model_pb_type == 'parsing':
        serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(features)
    elif params.model_pb_type == 'raw':
        serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(features)

    return model.export_savedmodel(params.model_pb, serving_input_receiver_fn)


def model_save_pb_dssm(params, model):
    features = {
        'user_cate_feats': tf.FixedLenFeature([params.user_cate_field_count], tf.int64),
        'item_cont_feats': tf.FixedLenFeature([params.item_cont_field_count], tf.float32),
        'item_cate_feats': tf.FixedLenFeature([params.item_cate_field_count], tf.int64)
    }
    for field_name in params.multi_cate_field_list:
        features[field_name[0]] = tf.VarLenFeature(tf.int64)

    if params.model_pb_type == 'parsing':
        serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(features)
    elif params.model_pb_type == 'raw':
        serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(features)

    return model.export_savedmodel(params.model_pb, serving_input_receiver_fn)


In [3]:
# -*- coding:utf-8 -*-
'''
功能: 1.通用工具类函数 2.解析配置文件lr.conf&dnn.conf
'''
import datetime
import os
import tensorflow as tf


def arg_parse(argv):
    parse_dict = dict()
    for i in range(1, len(argv)):
        line_parse = argv[i].split("=")
        key = line_parse[0].strip()
        value = line_parse[1].strip()
        parse_dict[key] = value
    return parse_dict


def str_list_2_int_list(str_list):
    int_list = list()
    for i in str_list:
        if type(i) == str:
            int_list.append(int(i))
    return int_list


def str_list_2_float_list(str_list):
    float_list = list()
    for i in str_list:
        if type(i) == str:
            float_list.append(float(i))
    return float_list

def parse_feats_conf(conf_path, alg_name):  # 解析配置文件
    cont_field_count = 0
    cate_field_count = 0
    multi_cate_field_count = 0
    multi_cate_field_list = list()  # [(multi_cate_feat_name, topN)]
    total_field_count = 0
    # dssm
    user_cont_field_count = 0
    user_cate_field_count = 0
    item_cont_field_count = 0
    item_cate_field_count = 0
    # target-attention
    target_att_alg_name = ['din', 'dinfm']
    target_att_item_single = dict()  # {attention-id: item-index}  single_cate_feats(e.g. item_cat1) was merged into one column named 'cate_feats', so we need record the index
    target_att_item = dict()  # {attention-id: item-name} multi_cate_feats(e.g. item_tags) are independent columns, so it is enough to record column names
    target_att_user = dict()  # {attention-id: user-name} multi_cate_feats(e.g. user_click_cat1) are independent columns, so it is enough to record column names
    target_att_1vN_list = list()  # target-attention: 1-n [[user-name1, item-index1], [user-name2, item-index2], ...]
    target_att_NvN_list = list()  # target-attention: n-n [[user-name1, item-name1], [user-name2, item-name2], ...]

    files = os.listdir(conf_path)
    for file in files:
        print (file)
        if file != "dnn.conf" and file != "lr.conf":
            continue
        file_path = conf_path + "/" + file
        with open(file_path, 'r') as f:
            for line in f.readlines():
                line_data = line.strip()
                if line_data == '':
                    continue
                try:
                    config_arr = line_data.split("\t")
                    col_name = config_arr[0]
                    new_col_name = config_arr[1]
                    if new_col_name == "none":
                        new_col_name = col_name
                    result_type = config_arr[2]

                    is_drop = int(config_arr[8])
                    att_id = config_arr[10]  # attention_id

                    if is_drop == 1:
                        continue

                    total_field_count += 1

                    if result_type == 'arr':  # multi_cate
                        func_parse = config_arr[5]  # pre_parse_func = config_arr[4]
                        result_parse = config_arr[7]  # result_parse_func = config_arr[6]
                        # target-attention
                        if alg_name in target_att_alg_name and att_id != '0':
                            feat_type = col_name.split('_')[0]
                            if feat_type == "item":
                                target_att_item[att_id] = new_col_name
                            else:  # "user"
                                target_att_user[att_id] = new_col_name

                        # multi_cate_feats
                        top_n = result_parse if func_parse == 'none' else func_parse
                        multi_cate_field_list.append((new_col_name, int(top_n.split("=")[1])))
                        multi_cate_field_count += 1

                    elif result_type == 'string':  # single_cate
                        # target-attention
                        if alg_name in target_att_alg_name and att_id != '0':
                            feat_type = col_name.split('_')[0]
                            if feat_type == "item":
                                target_att_item_single[att_id] = cate_field_count
                        # dssm
                        if alg_name == 'dssm':
                            feat_type = col_name.split('_')[0]
                            if feat_type == "item" and '#' not in col_name:  # '#' not in col_name是为了过滤user和item的交叉特征'#'
                                item_cate_field_count += 1
                            if feat_type == "user" and '#' not in col_name:
                                user_cate_field_count += 1
                        # single_cate_feats
                        cate_field_count += 1

                    elif result_type == 'float':  # cont
                        # dssm cont
                        if alg_name == 'dssm':
                            feat_type = col_name.split('_')[0]
                            if feat_type == "item":
                                item_cont_field_count += 1
                            if feat_type == "user":
                                user_cont_field_count += 1
                        # cont
                        cont_field_count += 1
                    else:
                        print("%s is error!!!" % line_data)
                except Exception as e:
                    print("-----------feat_conf is Error!!!!-----------")
                    print(e)
                    print(line_data)
                    exit(-1)

    parse_feats_dict = dict()
    parse_feats_dict["cont_field_count"] = cont_field_count
    parse_feats_dict["cate_field_count"] = cate_field_count
    parse_feats_dict["multi_cate_field_count"] = multi_cate_field_count
    parse_feats_dict["multi_cate_field_list"] = multi_cate_field_list
    parse_feats_dict["total_field_count"] = total_field_count
    # dssm
    parse_feats_dict["user_cont_field_count"] = user_cont_field_count
    parse_feats_dict["user_cate_field_count"] = user_cate_field_count
    parse_feats_dict["item_cont_field_count"] = item_cont_field_count
    parse_feats_dict["item_cate_field_count"] = item_cate_field_count
    # target-attention
    for k, v in target_att_user.items():
        if k in target_att_item_single.keys():
            target_att_1vN_list.append((v, target_att_item_single[k]))  # e.g. [(user_click_cat1, 3), ...], '3' means 'item_cat1' is the third feat of 'cate_feats'
    parse_feats_dict["target_att_1vN_list"] = target_att_1vN_list
    for k, v in target_att_user.items():
        if k in target_att_item.keys():
            target_att_NvN_list.append((v, target_att_item[k]))  # e.g. [(user_click_tags: item_tags), ...], both are multi_feats
    parse_feats_dict["target_att_NvN_list"] = target_att_NvN_list

    return parse_feats_dict

In [4]:
# -*- coding:utf-8 -*-
'''
功能：加载tfrecord
-------
所有连续特征合并为features["cont_feats"], 所有单值离散特征合并为features["cate_feats"], 此时由于位置和特征一一对应, 故必须补齐缺失值, 即数据类型为dense类型->FixedLenFeature
多值离散特征不合并, 各自为一维特征(features["name1"], features["name2"], ..., features["nameN"])
-------
由于多值离散特征fetures["name*"]最终往往会进行sum_pool或mean_pool处理, 故可以不补齐缺失值, 即sparse类型->VarLenFeature, 配合embedding_lookup_sparse使用即可
当然多值离散特征也可以像连续特征/单值离散特征一样, 补齐为dense类型->FixedLenFeature, 配合embedding_lookup和reduce_sum/reduce_mean即可
但多值离散特征往往维度较大且稀疏, 建议使用sparse方式以节省空间与时间, 除了一些特殊情况(如din)
-------
关于使用了target-attention的din与dinfm模型, 多值离散特征用target-attention机制代替了简单的sum/mean-pooling, 所以不能使用embedding_lookup_sparse, 即不支持sparse数据
这里使用pdded_batch实现了'sparse2dense', 但还是建议在生成tfrecord数据之前直接利用spark补齐缺失值生成dense数据, spark效率更高
-------
'''
import tensorflow as tf


def parse_example(example, params):
    features = {
        'label': tf.FixedLenFeature([1], tf.float32),
        'cont_feats': tf.FixedLenFeature([params.cont_field_count], tf.float32),
        'cate_feats': tf.FixedLenFeature([params.cate_field_count], tf.int64),
    }
    if params.multi_feats_type == 'dense':
        for name_topN in params.multi_cate_field_list:
            features[name_topN[0]] = tf.FixedLenFeature([name_topN[1]], tf.int64)
    elif params.multi_feats_type in ['sparse', 'sparse2dense']:
        for name_topN in params.multi_cate_field_list:
            features[name_topN[0]] = tf.VarLenFeature(tf.int64)

    parsed_features = tf.parse_single_example(example, features)
    label = parsed_features['label']
    parsed_features.pop('label')

    if params.multi_feats_type == 'sparse2dense':  # 配合padded_batch使用
        for name_topN in params.multi_cate_field_list:
            parsed_features[name_topN[0]] = tf.sparse_tensor_to_dense(parsed_features[name_topN[0]])

    return parsed_features, label


def input_fn(data_path, params):
    data_set = tf.data.TFRecordDataset(data_path)

    if params.multi_feats_type in ['dense', 'sparse']:
        data_set = data_set.map(lambda x: parse_example(x, params), num_parallel_calls=params.num_threads) \
            .batch(params.batch_size) \
            .repeat(params.epochs) \
            .prefetch(params.batch_size)

    elif params.multi_feats_type == 'sparse2dense':  # 利用padded_batch实现sparse数据转为dense
        pad_shapes = dict()
        pad_shapes_label = tf.TensorShape([1])
        pad_shapes["cont_feats"] = tf.TensorShape([params.cont_field_count])
        pad_shapes["cate_feats"] = tf.TensorShape([params.cate_field_count])
        for name_topN in params.multi_cate_field_list:
            pad_shapes[name_topN[0]] = tf.TensorShape([name_topN[1]])
        pad_shapes = (pad_shapes, pad_shapes_label)
        data_set = data_set.map(lambda x: parse_example(x, params), num_parallel_calls=params.num_threads) \
            .padded_batch(params.batch_size, padded_shapes=pad_shapes) \
            .repeat(params.epochs) \
            .prefetch(params.batch_size)

    else:
        print("multi_feats_type error!!!")
        exit(-1)

    iterator = data_set.make_one_shot_iterator()
    feature_dict, label = iterator.get_next()
    return feature_dict, label

In [5]:
# -*- coding:utf-8 -*-

'''
使用target-attention默认数据格式为dense
'''
import tensorflow as tf

def model_fn(labels, features, mode, params):
    tf.set_random_seed(2020)
    cont_feats = features["cont_feats"]
    cate_feats = features["cate_feats"]

    #shape=(?, 18),  shape=(?, 33)
    print (cont_feats)
    print (cate_feats)
    
    cont_feats_index = tf.Variable([[i for i in range(params.cont_field_count)]], trainable=False, dtype=tf.int64, name="cont_feats_index")
    # shape=(1, 18)
    cont_feats_index = tf.add(cont_feats_index, params.cate_emb_space_size)

    #9000018
    feats_size = params.cont_field_count + params.cate_emb_space_size
    
    # shape=(9000018, 32)
    feats_emb = emb_init(name='feats_emb', feat_num=feats_size, embedding_size=params.embedding_size)

    
    print (cont_feats_index)
    print (feats_size)
    print (feats_emb)
    
    # cont_feats
    #tensor("embedding_lookup:0", shape=(1, 18, 32), dtype=float32)
    #Tensor("Reshape:0", shape=(?, 18, 1), dtype=float32)
    #Tensor("Mul:0", shape=(?, 18, 32), dtype=float32)
    cont_emb = tf.nn.embedding_lookup(feats_emb, ids=cont_feats_index)  # None * F * embedding_size
    cont_value = tf.reshape(cont_feats, shape=[-1, params.cont_field_count, 1])
    embeddings = tf.multiply(cont_emb, cont_value)
    
    print (cont_emb)
    print (cont_value)
    print (embeddings)
    
    # cate_feats
    #Tensor("embedding_lookup_1:0", shape=(?, 33, 32), dtype=float32)
    #Tensor("concat_1:0", shape=(?, 51, 32), dtype=float32)
    cate_emb = tf.nn.embedding_lookup(feats_emb, ids=cate_feats)
    embeddings = tf.concat([embeddings, cate_emb], axis=1)
    
    print (cate_emb)
    print (embeddings)
    
    # multi_cate_feats
    for name_topN in params.multi_cate_field_list:
        multi_cate_emb = tf.nn.embedding_lookup(feats_emb, ids=features[name_topN[0]])  # None, topN, embedding_size
        multi_cate_emb = tf.reduce_sum(multi_cate_emb, axis=1)  # None, embedding_size
        multi_cate_emb = tf.reshape(multi_cate_emb, shape=[-1, 1, params.embedding_size])
        embeddings = tf.concat([embeddings, multi_cate_emb], axis=1)
        print ('name_topN ', name_topN)
        print (embeddings)
    # target-attention 1vN (e.g. item_cat1 & user_click_cat1)
    for k_v in params.target_att_1vN_list:
        item_feat = tf.split(cate_feats, params.cate_field_count, axis=1)[k_v[1]]
        user_feat = features[k_v[0]]
        nonzero_len = tf.count_nonzero(user_feat, axis=1)
        item_emb = tf.nn.embedding_lookup(feats_emb, ids=item_feat)  # [B, 1, H]
        item_emb = tf.reshape(item_emb, shape=[-1, params.embedding_size])  # [B, H]
        user_emb = tf.nn.embedding_lookup(feats_emb, ids=user_feat)  # [B, T, H])
        att_1vN_emb = attention(item_emb, user_emb, nonzero_len)  # [B, 1, H]
        embeddings = tf.concat([embeddings, att_1vN_emb], axis=1)
        print ('k_v ', k_v, item_feat)
        print (embeddings)
    # target-attention NvN (e.g. item_tags & user_click_tags)
    for k_v in params.target_att_NvN_list:
        item_feat = features[k_v[1]]
        user_feat = features[k_v[0]]
        nonzero_len = tf.count_nonzero(user_feat, axis=1)
        item_emb = tf.nn.embedding_lookup(feats_emb, ids=item_feat)  # [B, N, H]
        user_emb = tf.nn.embedding_lookup(feats_emb, ids=user_feat)  # [B, T, H])
        att_NvN_emb = attention_multi(item_emb, user_emb, nonzero_len)  # [B, 1, H]
        embeddings = tf.concat([embeddings, att_NvN_emb], axis=1)
        print ('k_v_1 ', k_v, item_feat)
        print (embeddings)
        
    # deep
    embeddings = tf.layers.flatten(embeddings)
    len_layers = len(params.hidden_units)
    for i in range(0, len_layers):
        embeddings = tf.layers.dense(inputs=embeddings, units=params.hidden_units[i], activation=tf.nn.relu)
    out = tf.layers.dense(inputs=embeddings, units=1)
    score = tf.identity(tf.nn.sigmoid(out), name='score')
    model_estimator_spec = model_optimizer(params, mode, labels, score)
    return model_estimator_spec


def model_estimator(params):
    # shutil.rmtree(conf.model_dir, ignore_errors=True)
    tf.reset_default_graph()
    config = tf.estimator.RunConfig() \
        .replace(
        session_config=tf.ConfigProto(device_count={'GPU': params.is_GPU, 'CPU': params.num_threads}, 
                                      gpu_options=tf.GPUOptions(allow_growth=True),
                                      inter_op_parallelism_threads=params.num_threads,
                                      intra_op_parallelism_threads=params.num_threads),
        log_step_count_steps=params.log_step_count_steps,
        save_checkpoints_steps=params.save_checkpoints_steps,
        keep_checkpoint_max=params.keep_checkpoint_max,
        save_summary_steps=params.save_summary_steps)

    model = tf.estimator.Estimator(
        model_fn=model_fn,
        config=config,
        model_dir=params.model_dir,
        params=params,
    )
    return model


In [6]:
# -*- coding:utf-8 -*-

import tensorflow as tf
import shutil

#################### CMD Arguments ####################
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
# ----------------common------------------
tf.app.flags.DEFINE_string("dt", "", "date")
tf.app.flags.DEFINE_string("alg_name", "din", "algorithm name")
tf.app.flags.DEFINE_string("task_mode", "train", "task mode type {train, eval, infer, debug}")
tf.app.flags.DEFINE_integer("epochs", 1, "epochs of training")
tf.app.flags.DEFINE_integer("batch_size", 8, "Number of batch size")
tf.app.flags.DEFINE_integer("embedding_size", 32, "Embedding size")
tf.app.flags.DEFINE_integer("cate_emb_space_size", 9000000, "emb space")
tf.app.flags.DEFINE_string("optimizer", "Adam", "optimizer type {Adam, Adagrad, GD, Momentum}")
tf.app.flags.DEFINE_float("learning_rate", 0.001, "learning rate")
tf.app.flags.DEFINE_integer("learning_rate_decay_steps", 10000000, "")
tf.app.flags.DEFINE_float("learning_rate_decay_rate", 0.9, "")
tf.app.flags.DEFINE_float("l2_reg", 0.01, "")
tf.app.flags.DEFINE_integer("log_step_count_steps", 100, "")
tf.app.flags.DEFINE_list("hidden_units", "512,256,256,128", "the layers of dnn")
tf.app.flags.DEFINE_list("dropout", "0.8,0.8,0.8,0.8", "dropout rate")
# ----------------device-----------------
tf.app.flags.DEFINE_integer("is_GPU", 0, "use GPU or not, 1->yes, 0->no")
tf.app.flags.DEFINE_integer("num_cpu", 1, "Number of CPU")
tf.app.flags.DEFINE_integer("num_threads", 1, "Number of threads")
# ----------------model_save--------------------
tf.app.flags.DEFINE_string("model_pb_type", "parsing", "model_pb export type {raw, parsing}")
tf.app.flags.DEFINE_string("model_pb", "data/model_save_pb/default", "the path for exporting model pb after training")
tf.app.flags.DEFINE_string("model_dir", "data/model_save_dir/default", "the path for saving model checkpoint between training")
tf.app.flags.DEFINE_boolean("clear_existing_model_dir", False, "clear existing model_dir or not")
tf.app.flags.DEFINE_integer("keep_checkpoint_max", 1, "")
tf.app.flags.DEFINE_integer("save_checkpoints_steps", 100, "")
tf.app.flags.DEFINE_integer("save_summary_steps", 500, "save summary every steps")
# ----------------data(tfrecord)--------------------
tf.app.flags.DEFINE_string("multi_feats_type", "sparse", "multi_cate_feats type {dense, sparse}")  # for notes, see utils/data_load.py
tf.app.flags.DEFINE_string("train_data", "data/tfrecord/demo/demo-sparse", "the path of train data")
tf.app.flags.DEFINE_string("eval_data", "data/tfrecord/demo/demo-sparse", "the path of eval data")
# ---------------parse features conf----------------
tf.app.flags.DEFINE_string("feats_conf", "data/tfrecord/demo/", "the path of feature config files")
parse_feats_dict = parse_feats_conf(FLAGS.feats_conf, FLAGS.alg_name)
print (parse_feats_dict)
tf.app.flags.DEFINE_integer("total_field_count", parse_feats_dict["total_field_count"], "")
tf.app.flags.DEFINE_integer("cont_field_count", parse_feats_dict["cont_field_count"], "")
tf.app.flags.DEFINE_integer("cate_field_count", parse_feats_dict["cate_field_count"], "")
tf.app.flags.DEFINE_integer("multi_cate_field_count", parse_feats_dict["multi_cate_field_count"], "")
tf.app.flags.DEFINE_list("multi_cate_field_list", parse_feats_dict["multi_cate_field_list"], "")
tf.app.flags.DEFINE_list("target_att_1vN_list", parse_feats_dict["target_att_1vN_list"], "")
tf.app.flags.DEFINE_list("target_att_NvN_list", parse_feats_dict["target_att_NvN_list"], "")
tf.app.flags.DEFINE_integer("user_cont_field_count", parse_feats_dict["user_cont_field_count"], "")
tf.app.flags.DEFINE_integer("user_cate_field_count", parse_feats_dict["user_cate_field_count"], "")
tf.app.flags.DEFINE_integer("item_cont_field_count", parse_feats_dict["item_cont_field_count"], "")
tf.app.flags.DEFINE_integer("item_cate_field_count", parse_feats_dict["item_cate_field_count"], "")
# ------------model special parameters-----------
# autoint
tf.app.flags.DEFINE_integer("autoint_layer_count", 2, "")
tf.app.flags.DEFINE_integer("autoint_emb_size", 16, "")
tf.app.flags.DEFINE_integer("autoint_head_count", 2, "")
tf.app.flags.DEFINE_integer("autoint_use_res", 1, "")


def handle_arguments():
    # ---common----
    FLAGS.hidden_units = str_list_2_int_list(FLAGS.hidden_units)  # python main.py --hidden_units=512,256,128
    FLAGS.dropout = str_list_2_float_list(FLAGS.dropout)  # python main.py --dropout=0.8,0.8,0.8
    # ---data(tfrecord)---
    if FLAGS.alg_name in ["din", "dinfm"] and FLAGS.multi_feats_type == "sparse":  # target-attention函数的输入是dense_tensor, 所以多值离散特征需补齐缺失值, 不支持sparse
        FLAGS.multi_feats_type = "sparse2dense"


def check_arguments():
    print("-----------check Arguments------------START")
    print("alg_name: ", FLAGS.alg_name)
    print("task_mode: ", FLAGS.task_mode)
    print("hidden_units: ", FLAGS.hidden_units)
    print("model_dir: ", FLAGS.model_dir)
    print("model_pb: ", FLAGS.model_pb)
    print("clear_existing_model_dir: ", FLAGS.clear_existing_model_dir)
    print("train_data: ", FLAGS.train_data)
    print("eval_data: ", FLAGS.eval_data)
    print("epochs: ", FLAGS.epochs)
    print("embedding_size: ", FLAGS.embedding_size)
    print("batch_size: ", FLAGS.batch_size)
    print("dropout: ", FLAGS.dropout)
    print("optimizer: ", FLAGS.optimizer)
    print("learning_rate: ", FLAGS.learning_rate)
    print("total_field_count: ", FLAGS.total_field_count)
    print("cont_field_count: ", FLAGS.cont_field_count)
    print("cate_field_count: ", FLAGS.cate_field_count)
    print("multi_cate_field_count: ", FLAGS.multi_cate_field_count)
    print("multi_cate_field_list('feats_name', topN): ", FLAGS.multi_cate_field_list)
    print("multi_feats_type: ", FLAGS.multi_feats_type)
    print("target_att_1vN_list('user_feats_name', single_cate_index): ", FLAGS.target_att_1vN_list)
    print("target_att_NvN_list('user_feats_name', item_feats_name): ", FLAGS.target_att_NvN_list)
    print("-----------check Arguments-------------END")


def main(_):
    handle_arguments()
    check_arguments()

    if FLAGS.clear_existing_model_dir and FLAGS.task_mode == 'train':
        try:
            shutil.rmtree(FLAGS.model_dir)
        except Exception as e:
            print(e, "at clear_existing_model_dir")
        else:
            print("existing model cleaned at %s" % FLAGS.model_dir)

    model = model_estimator(FLAGS)

    if FLAGS.task_mode == "train":
        
        model.train(input_fn=lambda: input_fn(FLAGS.train_data, FLAGS))
        model_op.model_save_pb(FLAGS, model)
        eval_result = model.evaluate(input_fn=lambda: data_load.input_fn(FLAGS.eval_data, FLAGS))
        print('eval-auc=%.5f\t loss=%.5f\t' % (eval_result['auc'], eval_result['loss']))

    elif FLAGS.task_mode == "eval":
        eval_result = model.evaluate(input_fn=lambda: data_load.input_fn(FLAGS.eval_data, FLAGS))
        print('eval-auc=%.5f\t loss=%.5f\t' % (eval_result['auc'], eval_result['loss']))

    elif FLAGS.task_mode == "infer":
        # preds = model.predict(input_fn=lambda: data_load.input_fn(FLAGS.eval_data, FLAGS), predict_keys=["item_embedding"])
        # f = open(FLAGS.test_data, "rb")
        # with open(FLAGS.infer_result, "w") as fo:
        #     for line, p in zip(f, preds):
        #         id = line.decode("utf-8").split(" ")[10]
        #         # print(p)
        #         # fo.write("%f\n" % (prob["embedding"]))
        #         emb = ','.join(["%.6f" % f for f in list(p["item_embedding"])])
        #         fo.write(id + "|" + id + "|" + emb + "\n")
        pass

    elif FLAGS.task_mode == "debug":  # make some fake data for debugging !!!开发中，暂时不可用
        pass

    else:
        print("task_mode Error!")


if __name__ == "__main__":
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()

demo-sparse
lr.conf
dnn.conf
demo-dense
{'cont_field_count': 18, 'cate_field_count': 33, 'multi_cate_field_count': 15, 'multi_cate_field_list': [('match_pos_tags', 10), ('match_pos_ru_tags', 10), ('match_pos_tag_ext', 15), ('match_pos_ru_tags_ext', 15), ('item_topic_keys', 10), ('item_tags_keys', 10), ('item_tags_ext_keys', 15), ('user_month_cat1_keys', 10), ('user_month_cat2_keys', 10), ('user_month_media_id_keys', 30), ('user_month_tags_keys', 50), ('ruCat1ScoreKeys', 10), ('ruCat2ScoreKeys', 10), ('ruMediaIdScoreKeys', 30), ('ruTagsScoreKeys', 50)], 'total_field_count': 66, 'user_cont_field_count': 0, 'user_cate_field_count': 0, 'item_cont_field_count': 0, 'item_cate_field_count': 0, 'target_att_1vN_list': [('ruCat1ScoreKeys', 2), ('ruCat2ScoreKeys', 3), ('ruMediaIdScoreKeys', 4)], 'target_att_NvN_list': [('ruTagsScoreKeys', 'item_tags_keys')]}
-----------check Arguments------------START
alg_name:  din
task_mode:  train
hidden_units:  [512, 256, 256, 128]
model_dir:  data/model_save

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:105: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 288000544 elements. This may consume a large amount of memory.
  num_elements)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


KeyboardInterrupt: 